In [1]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'elementary_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'cpu', 'random_seed': 123, 'data_configs': {'data_class': 'tinybig.data.elementary_function', 'data_parameters': {'name': 'elementary_function', 'train_batch_size': 100, 'test_batch_size': 100, 'equation_index': 14}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 2000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.003, 'weight_decay': 0.0001}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.999}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'cpu', 'name': 'reconciled_polynomial_network', 'depth': 2, 'depth_alloc': [1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_la

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  cpu ; random_seed:  123


In [3]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  47


In [4]:
print('loading dataset...')
data_loader = data_obj.load()
print(data_loader['str_equation'])

loading dataset...
E.14,14,f,arcsinh(x+y),2,x,0,0.5,y,0,0.5


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device,
                                     metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 0.2524417042732239, Test Score: 0.2524417055689922, Time Cost: 0.005294084548950195
Epoch: 100, Test Loss: 0.0001152791410277132, Test Score: 0.00011527914101987585, Time Cost: 0.004798173904418945
Epoch: 200, Test Loss: 2.3911261905595893e-05, Test Score: 2.391126250305909e-05, Time Cost: 0.004842042922973633
Epoch: 300, Test Loss: 6.5726183720471456e-06, Test Score: 6.572618162809303e-06, Time Cost: 0.0054819583892822266
Epoch: 400, Test Loss: 3.51954729467252e-06, Test Score: 3.519547253277029e-06, Time Cost: 0.0048449039459228516
Epoch: 500, Test Loss: 2.334105454337987e-06, Test Score: 2.3341054057180665e-06, Time Cost: 0.0048980712890625
Epoch: 600, Test Loss: 1.6119407689529907e-06, Test Score: 1.6119407674374046e-06, Time Cost: 0.005117177963256836
Epoch: 700, Test Loss: 1.1640622460618034e-06, Test Score: 1.1640622808364795e-06, Time Cost: 0.005037069320678711
Epoch: 800, Test Loss: 7.221749314112458e-07, Test Score: 7.221749273441866e-07

In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 1.5089853862074175e-09


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
